<a href="https://colab.research.google.com/github/devashishbotre/Autonomous-Lane-Detector/blob/main/scripts/Video_Processing_and_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

manideep1108_tusimple_path = kagglehub.dataset_download('manideep1108/tusimple')
devashishbotre_models_path = kagglehub.dataset_download('devashishbotre/models')

print('Data source import complete.')


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm

IMG_HEIGHT, IMG_WIDTH = 720, 1280  # Adjust to match your model's expected input size
NUM_CLASSES = 2  # Background and lane

In [ ]:
class RESA(nn.Module):
    def __init__(self, in_channels):
        super(RESA, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, 1, padding=0)
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.refine_conv = nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels)
        self.refine_bn = nn.BatchNorm2d(in_channels)
        self.shifts = [
            (1, 0), (2, 0), (3, 0),    # up-to-down
            (-1, 0), (-2, 0), (-3, 0), # down-to-up
            (0, -1), (0, -2), (0, -3), # right-to-left
            (0, 1), (0, 2), (0, 3)     # left-to-right
        ]
        distances = torch.tensor([(i**2 + j**2) for i, j in self.shifts], dtype=torch.float32)
        self.weights = torch.exp(-distances / (2 * 1.5**2))

    def forward(self, x):
        device = x.device
        self.weights = self.weights.to(device)
        contributions = []
        for (shift_h, shift_w), weight in zip(self.shifts, self.weights):
            shifted = torch.roll(x, shifts=(shift_h, shift_w), dims=(2, 3))
            contrib = self.relu(self.bn(self.conv(shifted)))
            contrib = self.relu(self.refine_bn(self.refine_conv(contrib)))
            contributions.append(contrib * weight)
        out = x + 0.5 * sum(contributions) / sum(self.weights)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.Up1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 2, 3, padding=1),
            nn.BatchNorm2d(in_channels // 2),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.Up2 = nn.Sequential(
            nn.Conv2d(in_channels // 2, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.smooth_conv = nn.Conv2d(out_channels, out_channels, 5, padding=2, groups=out_channels)
        self.smooth_bn = nn.BatchNorm2d(out_channels)
        self.thin1 = nn.Conv2d(in_channels // 2, in_channels // 2, 3, padding=1, groups=in_channels // 2)
        self.thin2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, groups=out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.Up1(x)
        x = x + torch.sigmoid(self.thin1(x)) * x
        x = self.Up2(x)
        x = x + torch.sigmoid(self.thin2(x)) * x
        x = self.relu(self.smooth_bn(self.smooth_conv(x)))
        return x

In [ ]:
class LaneNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, k_iterations=4):
        super(LaneNet, self).__init__()
        resnet = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        self.encoder = nn.Sequential(
            resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool,
            resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4
        )
        self.resa_layers = nn.ModuleList([RESA(512) for _ in range(k_iterations)])
        self.decoder = Decoder(512, 256)
        self.seg_head = nn.Conv2d(256, num_classes, 1)

    def forward(self, x):
        x = self.encoder(x)
        for resa in self.resa_layers:
            x = resa(x)
        x = self.decoder(x)
        seg_out = self.seg_head(x)
        seg_out = F.interpolate(seg_out, size=(IMG_HEIGHT, IMG_WIDTH), mode='bilinear', align_corners=True)
        seg_out = F.avg_pool2d(seg_out, kernel_size=3, padding=1, stride=1)
        return seg_out

In [ ]:
def overlay_mask(image, mask, alpha=0.5):
    mask_colored = np.zeros_like(image)
    mask_colored[mask == 1] = [0, 255, 0]  # Green for lanes
    overlay = cv2.addWeighted(image, 1.0, mask_colored, alpha, 0.0)
    return overlay


def numeric_sort_key(name):
    return int(os.path.splitext(name)[0])

# Process directory function
def process_directory(root_dir, model, device, output_dir="output_videos"):
    if not os.path.exists(root_dir):
        raise FileNotFoundError(f"Root directory {root_dir} does not exist")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize single video writers for all frames
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    original_video = cv2.VideoWriter(
        os.path.join(output_dir, "original_video_all.mp4"), fourcc, 20.0, (IMG_WIDTH, IMG_HEIGHT)
    )
    lane_video = cv2.VideoWriter(
        os.path.join(output_dir, "lane_detected_video_all.mp4"), fourcc, 20.0, (IMG_WIDTH, IMG_HEIGHT)
    )
    subdirs = sorted(
        [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))],
        key=lambda x: int(x)
    )

    total_frames_processed = 0

    for subdir in tqdm(subdirs, desc="Processing subdirectories"):
        subdir_path = os.path.join(root_dir, subdir)
        frames = sorted(
            [f for f in os.listdir(subdir_path) if f.endswith('.jpg')],
            key=numeric_sort_key
        )

        if len(frames) != 20:
            print(f"Warning: {subdir} has {len(frames)} frames, expected 20. Skipping.")
            continue

        for frame_name in frames:
            img_path = os.path.join(subdir_path, frame_name)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Error: Failed to load {img_path}. Skipping.")
                continue

            image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
            image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
            image_tensor = torch.from_numpy(image_rgb.transpose(2, 0, 1)).float() / 255.0
            image_tensor = image_tensor.unsqueeze(0).to(device)

            model.eval()
            with torch.no_grad():
                output = model(image_tensor)
                mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

            overlay_image = overlay_mask(image_resized, mask)
            original_video.write(image_resized)
            lane_video.write(overlay_image)
            total_frames_processed += 1

    # Release video writers after all subdirectories are processed
    original_video.release()
    lane_video.release()
    print(f"Generated single videos: 'original_video_all.mp4' and 'lane_detected_video_all.mp4'")
    print(f"Total frames processed: {total_frames_processed}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneNet(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load("/kaggle/input/models/lane_model_final (2).pth", map_location=device,weights_only=True))
model.eval()

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 179MB/s]


LaneNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
root_dir = "/kaggle/input/tusimple/TUSimple/test_set/clips/0530"  # Adjust to your directory
process_directory(root_dir, model, device)

Processing subdirectories: 100%|██████████| 1248/1248 [34:28<00:00,  1.66s/it]

Generated single videos: 'original_video_all.mp4' and 'lane_detected_video_all.mp4'
Total frames processed: 24960
